# STA 208: Homework 3 (Do not distribute)

__Instructions:__ Submit it on canvas.  The canvas should include all of your code either in this notebook file, or a separate python file that is imported and ran in this notebook.  We should be able to open this notebook and run everything here by running the cells in sequence.  The written portions can be either done in markdown and TeX in new cells or written clearly by hand when you hand it in.  Submit each file separately.

- Code should be well organized and documented
- All math should be clear and make sense sequentially
- When in doubt explain what is going on
- You will be graded on correctness of your math, code efficiency and succinctness, and conclusions and modelling decisions

__Exercise 1__ (10 pts)

Recall that surrogate losses for large margin classification take the form, $\phi(y_i x_i^\top \beta)$ where $y_i \in \{-1,1\}$ and $\beta, x_i \in \mathbb R^p$.

The following functions are used as surrogate losses for large margin classification.  Demonstrate if they are convex or not, and follow the instructions.

1. exponential loss: $\phi(x) = e^{-x}$
1. truncated quadratic loss: $\phi(x) = (\max\{1-x,0\})^2$
1. hinge loss: $\phi(x) = \max\{1-x,0\}$
1. sigmoid loss: $\phi(x) = 1 - \tanh(\kappa x)$, for fixed $\kappa > 0$
1. Plot these as a function of $x$.

(This problem is due to notes of Larry Wasserman.) 


__Exercise 2__ (20 pts)

Consider the truncated quadratic loss from (1.1.2).  For brevity let $a_+ = max\{a,0\}$ denote the positive part of $a$.

$$\ell(y_i,x_i,\beta) = \phi(y_i x_i^\top \beta) = (1-y_i x_i^\top \beta)_+^2$$

1. Consider the empirical risk, $R_n$ (the average loss over a training set) for the truncated quadratic loss.  What is gradient of $R_n$ in $\beta$?  Does it always exists?
1. Demonstrate that the gradient does not have continuous derivative everywhere.
1. Recall that support vector machines used the hinge loss $(1 - y_i x_i^\top \beta)_+$ with a ridge regularization.  Write the regularized optimization method for the truncated quadratic loss, and derive the gradient of the regularized empirical risk.
1. In quasi-Newton methods a matrix ($Q$) that is a surrogate for the Hessian of the objective $L$ is used to determine step direction.

$$
\beta \gets \beta - Q^{-1} \nabla L(\beta)
$$


Because the loss does not have continuous Hessian, instead of the Newton method, we will use a quasi-Newton method that replaces the Hessian with a quasi-Hessian (another matrix that is meant to approximate the Hessian).  Consider the following quasi-Hessian of the regularized objective to be $$G(\beta) = \frac 1n \sum_i 2 (x_i x_i^\top 1\{ y_i x_i^\top \beta < 1 \}) + 2 \lambda I.$$  Demonstrate that the quasi-Hessian is positive definite, and write pseudo-code for quasi-Newton optimization, comment on the computational complexity of this method.

**Exercise 3 (20 pts)**

Consider the simulation below.

1. Implement minibatch stochastic gradient descent using the truncated quadratic loss.  Access the data by iteratively calling the ``sim_data`` method below.  

2. With minibatch size of $1$ (SGD).  Vary to learning schedule to be constant, decaying with $\eta_t \propto t**{-1/2}$, and $\eta_t \propto t^{-1}$.  Compare with normal noise (the ``noise_dis`` parm).

3. Vary the minibatch size to see the change in performance, with the best learning schedule from 2. When you compare two methods, make sure that you compare them with the same amount of data accessed (so use 1:10 ratio of iterations if you are comparing a minibatch ratio of 10:1).

4. Redo 2, 3 with ``noise_dis`` set to ``"chisquare"``.

In [1]:
import numpy as np

In [6]:
class DataSimulator:
    """
    Simulate the data for linear classification
    """
    def __init__(self,p,noise_dist = "normal"):
        self.beta = np.random.normal(0,1,p)
        self.noise_dist = noise_dist
        self.p = p
        
    def sim_data(self,m = 1):
        p = self.p
        X = np.random.normal(0,1,(m,p))
        if self.noise_dist == "normal":
            eps = np.random.normal(0,1,m)
        if self.noise_dist == "chisquare":
            eps = np.random.chisquare(1,m)
        z = X @ self.beta + eps
        y = 1*(z > 0)
        return X, y

In [9]:
ds = DataSimulator(10)
ds.sim_datum(m=10)

(array([[-8.82760716e-01,  3.11498912e-01,  9.23795147e-01,
          1.61536818e+00,  8.96260892e-01,  3.97902536e-01,
         -7.26042790e-01, -8.15414265e-02,  2.40711842e-01,
          1.54514937e+00],
        [-9.09026409e-01,  1.84392586e-01,  7.11932756e-01,
          4.58398533e-01, -2.68785064e+00,  4.01885117e-01,
         -2.08945052e-01, -3.85312070e-04, -8.22808981e-01,
         -1.28058200e+00],
        [ 1.09881842e+00,  8.59804799e-01, -4.74453657e-01,
          3.58426865e-01,  9.96840138e-01,  1.36254269e+00,
          5.00507551e-01, -3.15185392e-01,  1.46405671e-01,
          7.70887297e-01],
        [-4.39329053e-01,  4.72952643e-02, -1.75216771e+00,
         -6.00553048e-01,  1.87289087e-01, -5.59439958e-01,
         -6.25566297e-01,  1.33319438e-01, -1.05928270e+00,
          5.15527106e-01],
        [ 7.92426056e-01, -5.14292065e-01, -3.97204403e-01,
         -4.49388093e-02,  1.34709454e-01,  4.26510677e-01,
          1.39772818e-01, -2.67506249e-01,  7.859055

__Exercise 4.__ (50 pts) 

Text data can be converted into vector data through a vectorization operation.  A corpus is a collection of documents and the dictionary is all of the words in the corpus.  Bag-of-words models will treat each document as a set of words, ignoring the order of the words.  Then a simple vectorizer will let $X_{i,j}$ be the number of times the $j$th word is in the $i$th document.  Two vectorizers are ``sklearn.feature_extraction.text.CountVectorizer`` and ``sklearn.feature_extraction.text.TfidfVectorizer``.

Below is an import of a reuters dataset.  I have written a def to process a single file.  Construct a response variable that has three categories, if the topic is 'earn', 'acq', or another category.  Import all of the data and construct two sparse vectorized matrices---look at ``scipy.sparse``---based on the two above vectorizations.  Use sklearn svm.SVC on the TRAIN split and predict on the TEST split.  Plot your ROC and PR curves for predicting 'earn' (versus everything else); tune the kernel and C parameters.  Do the same for predicting 'acq' versus everything else.  Write a paragraph summarizing the performance and tuning.

In [4]:
from lxml import html, etree

In [13]:
reu = html.parse("reuters/reut2-000.sgm") #You will have to do this for all sgm files here

In [6]:
import nltk
nltk.download()
# Download Corpora -> stopwords, Models -> punkt

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


In [7]:
def parse_reu(reu):
    """Parses the etree object and returns a list of dictionary of reuters attr
    Output: {'topics': the topic of the article, 'places': where it is located, 
        'split': training/test split, 'body':the text of the article as a set of words with stopwords removed}
    """
    root= reu.getroot()
    articles = root.body.getchildren()
    stop_words = set(stopwords.words('english'))
    reu_pl = []
    for a in articles:
        reu_parse = {}
        if a.attrib['topics'] != 'YES':
            next
        topics = a.find('topics').findall('d')
        if topics:
            reu_parse['topics'] = [t.text for t in topics]
        else:
            reu_parse['topics'] = []
        places = a.find('places').findall('d')
        if places:
            reu_parse['places'] = [t.text for t in places]
        reu_parse['split'] = a.attrib['lewissplit']
        rtxt = a.find('text')
        word_tokens = word_tokenize(rtxt.text_content())
        filtered_sentence = set([w.lower() for w in word_tokens if not w in stop_words])
        reu_parse['body'] = filtered_sentence
        reu_pl.append(reu_parse)
    return reu_pl

In [8]:
reu_pl = parse_reu(reu)

In [12]:
print(reu_pl[0]['topics'])
" ".join(reu_pl[0]['body'])

['cocoa']


'review sept lower 60 155,221 bean 4,350 week in may still held 5.81 kilos harvesting expected convertible 1986/87 much destinations making cumulative showers farmers 340 per u.s. prospects mln 2,400 27 july june/july routine comissaria delivered registered 4,450 final bags 1,780 dec crop early hundred april/may rose uruguay temporao 350 coming improving 1,750 785 thousand included fit oct/dec went aug butter late available reuter currency 6.13 ends part booked seems 1987/88 quality dificulties 753 export old february superior+ the 1,875 +bahia drought weeks consignment 4,400 view again 35 2,375 last - 1.25 ports selling 22 experiencing . commission middlemen march 1,880 feb tonne 4,351 currently times 6.2 levels throughout come arroba light , practically shipment 28 doubts continued recent new with there 2.28 restored trade standing would 4,340 sales prices york period since dlrs 2,380 doubt salvador zone nearby 4,415 january earlier 2,325 1.06 26 named cake going said 15 buyers sold 